<a href="https://colab.research.google.com/github/dnguyend/rayleigh_newton/blob/master/colab/BPairsUnitaryEigenTensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\newcommand{\cT}{\mathcal{T}}$
$\newcommand{\cB}{\mathcal{B}}$
$\newcommand{\C}{\mathbb{C}}$
$\newcommand{\R}{\mathbb{R}}$
# This workbook compute all eigentensor pairs by Rayleigh quotient iteration for a generalized Tensor eigenvalue problem  ($\cB$-tensor).
* Eigenpairs  
$$\cT(X^{[m-1]}) =\lambda \cB(X^{[d-1]}) $$
$\cT(X^{[m-1]})$ is a vector-valued function, from $\C^n$ or $\R^n$ to itself,  each entry is homogeneous of order $m-1$. We also use the notation $\cT(I, X^{[m-1]})= \cT(X^{[m-1]})$
The square bracket means the number of times $X$ is repeated.
* $\cT$ is a $(m, n)$ tensor, $\cB$ is a $(d, n)$ tensor. Both with coefficients in $\R$. However $X$ could be in $\C$.
* H-eigenpair is when $\cB(X^{[m]})=\sum x_i^m$ if $X=[x_0,\cdots x_n]$.

* Use generalized Rayleigh quotient method.

* Consider the complex case, $X\in \C^n$. We will normalize the result to the unitary constraint $X^*X = 1$. However, to have a uniform treatment we will use a linear constraint $C^TX=1$ for a vector $C$.

* We use the left inverse is $\frac{1}{|\cB(I, X^{[d-1]})|^2}\cB(I, X^{[d-1]})^*$. The Rayleigh quotient is
$$\frac{1}{|\cB(I, X^{[d-1]})|^2}\cB(I, X^{[d-1]})^*\cT(I, X^{[m-1]})$$

The eigenvalue count is $\sum_{i=0}^{n-1} (m-1)^i(d-1)^{n-i-1}$,
 which is $n(m-1)^{n-1}$ if $m=d$, and $\frac{(m-1)^n-(d-1)^n} {m-d}$ otherwise

# We confirm the eigenpairs in examples 4.3 and 4.10 of [Cui et. al (2014)].
CUI , C.-F., DAI , Y.-H. & NIE , J. (2014) All real eigenvalues of symmetric tensors. SIAM Journal on MatrixAnalysis and Applications, 35, 1582–1601.

# Some cells may be hidden. Please open in colab then unhide the cells


First, clone the project from github


In [1]:
!git clone https://github.com/dnguyend/rayleigh_newton

Cloning into 'rayleigh_newton'...
remote: Enumerating objects: 291, done.
remote: Counting objects: 100% (291/291), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 291 (delta 153), reused 267 (delta 138), pack-reused 0
Receiving objects: 100% (291/291), 14.81 MiB | 20.52 MiB/s, done.
Resolving deltas: 100% (153/153), done.


Import library and some codes to check output

In [2]:
import numpy as np
import numpy.linalg as la

from rayleigh_newton.core import utils as ut
from types import SimpleNamespace
from IPython.core.display import display    
from rayleigh_newton.core import display_tools as dt
from rayleigh_newton.core import linear_eigen_tensor_solver as le

In [3]:

def check_eig(T, B, all_eig):
    m = len(T.shape)
    d = len(B.shape)
    max_run = all_eig.lbd.shape[0]
    diff0 = np.empty(max_run)
    diff1 = np.empty(max_run)
    for i in range(max_run):
        Tnew = ut.tv_mode_product(T, all_eig.x[i, :], m-1)
        Bnew = ut.tv_mode_product(B, all_eig.x[i, :], d-1)
        lbdnew = np.sum(Bnew.conjugate()*Tnew)/la.norm(Bnew)**2
        diff0[i] = np.abs(lbdnew - all_eig.lbd[i])
        diff1[i] = la.norm(Tnew - lbdnew*Bnew)
    print('check lbd %f' % np.max(np.abs(diff0)))
    print('check equation %f' % np.max(np.abs(diff1)))

    diff3 = np.zeros(max_run-1)
    for i in range(max_run-1):
        if i > 0:
            factors = np.abs(np.abs(np.sum(all_eig.x[i, :][None, :]*all_eig.x[:i, :], axis=1)) - 1)
            diff3[i] = factors[np.argmax(factors)]
    print("check uniqueness")
    print(np.sort(diff3[1:][np.where(diff3 < 1e-1)]))    



A simple test. Generate a random tensor then run.

In [4]:
import numpy as np

n = 5
m = 6

T = ut.generate_symmetric_tensor(n, m)
B = ut.generate_symmetric_tensor(n, m)
C = np.random.randn(n)
max_itr = 200
print("case m=d=%d" % m)
x, lbd, cnt, converge, err = le.schur_form_B_tensor_rayleigh_linear(
    T, B, C, max_itr, delta=1e-8, x_init=None)

print(x, lbd, cnt, converge, err)

case m=d=6
[-0.23056726-0.24475009j -1.02126957+0.28583583j  0.0171179 -0.0308427j
  0.19545447-0.51422405j  0.4915945 +0.09723125j] (-0.25994655448224824+0.5205959563287005j) 25 True 2.3895600220792272e-15


One test with $m\neq d$

In [5]:
n = 4
m = 4
d = 3
T = ut.generate_symmetric_tensor(n, m)
B = ut.generate_symmetric_tensor(n, d)
C = np.random.randn(n)
max_itr = 200
print("case m=%d d=%d" % (m, d))
x, lbd, cnt, converge, err = le.schur_form_B_tensor_rayleigh_linear(
    T, B, C, max_itr, delta=1e-8, x_init=None)

print(x, lbd, cnt, converge, err)

n_eig = le.complex_eigen_cnt(m, d, n)
print("expecting  %d eigenvalues" % le.complex_eigen_cnt(m, d, n))
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=np.zeros((n_eig), dtype=bool),
    is_real=np.zeros((n_eig), dtype=bool))

all_eig, jcnt = le.find_all_unitary_eigenpair(
        all_eig, 0, T, B, max_itr=200, max_test=int(5e4), tol=1e-6, disc=2e-3)

print('Total %d' % jcnt)
check_eig(T, B, all_eig)
display(dt.display_all_real(all_eig).sort_values(by='lbd', ascending=False))


case m=4 d=3
[ 0.2156847 -0.24307046j -0.07896498+0.08603464j -0.27460078+0.07924125j
 -0.31502831-0.14764599j] (-0.5158617407246446+0.8124112728170584j) 9 True 7.076311083754595e-17
expecting  65 eigenvalues
Found 1 eigenpairs
Found 3 eigenpairs
Found 5 eigenpairs
Found 7 eigenpairs
Found 9 eigenpairs
Found 10 eigenpairs
Found 12 eigenpairs
Found 14 eigenpairs
Found 15 eigenpairs
Found 17 eigenpairs
Found 19 eigenpairs
Found 20 eigenpairs
Found 21 eigenpairs
Found 23 eigenpairs
Found 25 eigenpairs
Found 27 eigenpairs
Found 29 eigenpairs
Found 30 eigenpairs
Found 31 eigenpairs
Found 33 eigenpairs
Found 35 eigenpairs
Found 37 eigenpairs
Found 39 eigenpairs
Found 40 eigenpairs
Found 42 eigenpairs
Found 43 eigenpairs
Found 45 eigenpairs
Found 47 eigenpairs
Found 49 eigenpairs
Found 51 eigenpairs
Found 53 eigenpairs
Found 55 eigenpairs
Found 56 eigenpairs
Found 58 eigenpairs
Found 60 eigenpairs
Found 61 eigenpairs
Found 63 eigenpairs
Total 65
check lbd 0.000000
check equation 0.000000
chec

,lbd,0,1,2,3
60,3.730226,-0.143382,0.755065,-0.499272,-0.400057
29,2.044319,0.165814,0.539525,0.825183,-0.022186
9,1.751733,0.137152,0.343929,0.914848,-0.161105
14,1.501820,0.648492,-0.185312,0.731862,0.097445
30,1.480497,-0.885126,0.265480,0.312933,0.219420
55,1.346738,0.810421,-0.402215,0.423208,-0.048332
42,0.914629,-0.109736,0.752973,-0.281749,-0.584472
0,0.502411,-0.228380,0.240932,0.425816,-0.841710
39,0.181340,-0.283257,0.280936,0.607164,-0.687162
20,0.154222,-0.463811,-0.173855,0.862762,-0.101467


# Example 4.3 in [Cui et al. 2014]
CUI , C.-F., DAI , Y.-H. & NIE , J. (2014) All real eigenvalues of symmetric tensors. SIAM Journal on MatrixAnalysis and Applications, 35, 1582–1601.

Generate the tensor, compare the tensor and the polynomial, then run

In [6]:
import sympy as sp
n = 3
m = 4
x0, x1, x2 = sp.symbols('x0 x1 x2')
XX = sp.Matrix([x0, x1, x2])
a = 3
P = 2*x0**4 + 3*x1**4 + 5*x2**4 + 4*a*x0**2*x1*x2
T = ut.generate_symmetric_tensor_from_poly(XX, P)
X = np.random.randn(3)
print(P.subs([(XX[i], X[i]) for i in range(3)]))
print(ut.tv_mode_product(T, X, 4))

0.371400684645398
0.3714006846453983


In [7]:
B = np.zeros(m*[n])
for i in range(n):
    B[tuple(m*[i])] = 1

n_eig = le.complex_eigen_cnt(m, m, n)
print("expecting  %d eigenvalues" % le.complex_eigen_cnt(m, m, n))
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=np.zeros((n_eig), dtype=bool),
    is_real=np.zeros((n_eig), dtype=bool))

all_eig, jcnt = le.find_all_unitary_eigenpair(
        all_eig, 0, T, B, max_itr=200, max_test=int(4e3), tol=1e-6, disc=3e-3)  
print('Total %d' % jcnt)
check_eig(T, B, all_eig)
display(dt.display_all_real(all_eig).sort_values(by='lbd', ascending=False))


expecting  27 eigenvalues
Found 1 eigenpairs
Found 2 eigenpairs
Found 3 eigenpairs
Found 5 eigenpairs
Found 6 eigenpairs
Found 7 eigenpairs
Found 8 eigenpairs
Found 9 eigenpairs
Found 11 eigenpairs
Found 13 eigenpairs
Found 15 eigenpairs
Found 17 eigenpairs
Found 19 eigenpairs
Total 19
check lbd 8.139541
check equation 0.000000
check uniqueness
[0.99999856]


,lbd,0,1,2
6,7.450520,5.936946e-01,-5.251851e-01,-6.096780e-01
7,7.450520,-5.936946e-01,-5.251851e-01,-6.096780e-01
0,5.000000,-9.354771e-17,-1.462951e-06,-1.000000e+00
1,3.000000,7.602497e-17,1.000000e+00,-2.089903e-08
8,2.000000,1.000000e+00,4.317495e-21,4.966973e-20
5,-1.395156,-6.841141e-01,5.393149e-01,-4.910472e-01
2,-1.395156,-6.841141e-01,-5.393149e-01,4.910472e-01


The previous example seems to have only 19 complex pairs, the real pairs match with the cited paper.

# Example 4.10 Test the Motzkin polynomial

In [8]:
n = 3
m = 6
x0, x1, x2 = sp.symbols('x0 x1 x2')
XX = sp.Matrix([x0, x1, x2])

P = x0**4*x1**2 + x0**2*x1**4 + x2**6 - 3*x0**2*x1**2*x2**2
T = ut.generate_symmetric_tensor_from_poly(XX, P)
X = np.random.randn(3)
print(P.subs([(XX[i], X[i]) for i in range(n)]))
print(ut.tv_mode_product(T, X, m))

B = np.zeros(m*[n])
for i in range(n):
    B[tuple(m*[i])] = 1

n_eig = le.complex_eigen_cnt(m, m, n)
print("expecting  %d eigenvalues" % le.complex_eigen_cnt(m, m, n))
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=np.zeros((n_eig), dtype=bool),
    is_real=np.zeros((n_eig), dtype=bool))

all_eig, jcnt = le.find_all_unitary_eigenpair(
        all_eig, 0, T, B, max_itr=200, max_test=int(1e4), tol=5e-5, disc=1e-3)  


7.05317620611943
7.053176206119429
expecting  75 eigenvalues
Found 1 eigenpairs
Found 2 eigenpairs
Found 3 eigenpairs
Found 4 eigenpairs
Found 5 eigenpairs
Found 6 eigenpairs
Found 7 eigenpairs
Found 9 eigenpairs
Found 10 eigenpairs
Found 11 eigenpairs
Found 12 eigenpairs
Found 13 eigenpairs
Found 15 eigenpairs
Found 17 eigenpairs
Found 19 eigenpairs
Found 21 eigenpairs
Found 23 eigenpairs
Found 24 eigenpairs
Found 25 eigenpairs
Found 26 eigenpairs
Found 27 eigenpairs
Found 29 eigenpairs
Found 31 eigenpairs
Found 33 eigenpairs
Found 35 eigenpairs
Found 37 eigenpairs
Found 39 eigenpairs
Found 41 eigenpairs
Found 42 eigenpairs
Found 44 eigenpairs
Found 46 eigenpairs
Found 47 eigenpairs
Found 49 eigenpairs
Found 51 eigenpairs


Review output

In [9]:
print('Total %d' % jcnt)    
check_eig(T, B, all_eig)
display(dt.display_all_real(all_eig).sort_values(by='lbd', ascending=False))


Total 51
check lbd 1.732051
check equation 0.000000
check uniqueness
[0.72000783]


,lbd,0,1,2
4,1.000000e+00,2.045567e-05,-2.715425e-05,1.000000e+00
10,1.000000e+00,-7.071068e-01,-7.071068e-01,-1.203809e-13
26,1.000000e+00,7.071068e-01,-7.071068e-01,1.468029e-11
23,5.554044e-02,-7.718280e-01,-3.525470e-01,-5.291429e-01
11,5.554044e-02,3.525470e-01,7.718280e-01,5.291429e-01
41,5.554044e-02,7.718280e-01,-3.525470e-01,5.291429e-01
0,5.554044e-02,3.525470e-01,-7.718280e-01,-5.291429e-01
1,5.554044e-02,-7.718280e-01,-3.525470e-01,5.291429e-01
9,5.554044e-02,-3.525470e-01,7.718280e-01,-5.291429e-01
12,5.554044e-02,7.718280e-01,-3.525470e-01,-5.291429e-01



We do not find the full number $n(m-1)^{n-1)$. Apply a perturbation then compute eigen pairs again

In [10]:
Ta = T.copy()
eps = 1e-4
for i in range(n):
    Ta[m*[i]] += (i+1)*eps

# eps = 1e-3
# np.random.seed(0)
# Ta = T + eps*np.random.randn(*T.shape)
all_eig2 = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=np.zeros((n_eig), dtype=bool),
    is_real=np.zeros((n_eig), dtype=bool))
print("expecting  %d eigenvalues" % le.complex_eigen_cnt(m, m, n))
all_eig2, _ = le.find_all_unitary_eigenpair(
    all_eig2, 0, Ta, B, max_itr=200, max_test=int(1e4), tol=1e-6, disc=1e-3)  



expecting  75 eigenvalues
Found 1 eigenpairs
Found 3 eigenpairs
Found 4 eigenpairs
Found 6 eigenpairs
Found 7 eigenpairs
Found 8 eigenpairs
Found 10 eigenpairs
Found 11 eigenpairs
Found 13 eigenpairs
Found 15 eigenpairs
Found 17 eigenpairs
Found 19 eigenpairs
Found 20 eigenpairs
Found 22 eigenpairs
Found 24 eigenpairs
Found 25 eigenpairs
Found 27 eigenpairs
Found 28 eigenpairs
Found 29 eigenpairs
Found 30 eigenpairs
Found 32 eigenpairs
Found 34 eigenpairs
Found 35 eigenpairs
Found 37 eigenpairs
Found 39 eigenpairs
Found 41 eigenpairs
Found 42 eigenpairs
Found 43 eigenpairs
Found 44 eigenpairs
Found 46 eigenpairs
Found 47 eigenpairs
Found 48 eigenpairs
Found 50 eigenpairs
Found 51 eigenpairs
Found 53 eigenpairs
Found 54 eigenpairs
Found 56 eigenpairs
Found 58 eigenpairs
Found 59 eigenpairs
Found 61 eigenpairs
Found 63 eigenpairs
Found 65 eigenpairs
Found 67 eigenpairs
Found 69 eigenpairs
Found 71 eigenpairs
Found 73 eigenpairs


In [11]:
all_eig3 = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=np.zeros((n_eig), dtype=bool),
    is_real=np.zeros((n_eig), dtype=bool))
eig_cnt = 0
for j in range(all_eig2.lbd.shape[0]):
    bad = False
    if not np.isnan(all_eig2.lbd[j]):
        try:
            C = np.random.randn(n)
            x, lbd, ctr, converge, err =\
                le.schur_form_B_tensor_rayleigh_linear(
                    T, B, C, max_itr=200, delta=5e-5, x_init=all_eig2.x[j, :])

        except Exception as e:
            print(e)
            bad = True
    if not bad:
        old_eig = eig_cnt
        if (err < 1e-6):
            eig_cnt = le.insert_eigen(all_eig3, x, lbd, eig_cnt, m, m, tol=1e-6, disc=1e-3)
        if (eig_cnt > old_eig):
            print('Found %d eigenpairs' % eig_cnt)
            
print('Total %d' % jcnt)
check_eig(Ta, B, all_eig3)
display(dt.display_all_real(all_eig3).sort_values(by='lbd', ascending=False))    

Found 1 eigenpairs
Found 2 eigenpairs
Found 3 eigenpairs
Found 4 eigenpairs
Found 5 eigenpairs
Found 6 eigenpairs
Found 8 eigenpairs
Found 10 eigenpairs
Found 11 eigenpairs
Found 13 eigenpairs
Found 14 eigenpairs
Found 15 eigenpairs
Found 16 eigenpairs
Found 17 eigenpairs
Found 19 eigenpairs
Found 20 eigenpairs
Found 22 eigenpairs
Found 23 eigenpairs
Found 25 eigenpairs
Found 26 eigenpairs
Found 28 eigenpairs
Found 29 eigenpairs
Found 31 eigenpairs
Found 33 eigenpairs
Singular matrix
Found 35 eigenpairs
Found 37 eigenpairs
Found 39 eigenpairs
Found 41 eigenpairs
Found 43 eigenpairs
Found 45 eigenpairs
Found 47 eigenpairs
Found 49 eigenpairs
Total 51
check lbd nan
check equation nan
check uniqueness
[0.47085628]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in cdouble_scalars
  # This is added back by InteractiveShellApp.init_path()


,lbd,0,1,2
1,1.000000e+00,-7.795135e-24,1.117503e-06,1.000000e+00
25,1.000000e+00,-7.071068e-01,-7.071068e-01,-1.154124e-12
14,1.000000e+00,-7.071068e-01,7.071068e-01,0.000000e+00
10,5.554044e-02,7.718280e-01,-3.525470e-01,-5.291429e-01
2,5.554044e-02,3.525470e-01,-7.718280e-01,5.291429e-01
19,5.554044e-02,7.718280e-01,3.525470e-01,5.291429e-01
13,5.554044e-02,3.525470e-01,7.718280e-01,-5.291429e-01
22,5.554044e-02,7.718280e-01,3.525470e-01,-5.291429e-01
0,5.554044e-02,-3.525470e-01,-7.718280e-01,-5.291429e-01
15,5.554044e-02,3.525470e-01,-7.718280e-01,-5.291429e-01


Thus, it seems there are 51 distinct pairs, and there are a number of pairs with multiplicity $>1$, similar to the Z-Pair case.

# A few more runs

In [12]:
n = 3
m = 5
T = ut.generate_symmetric_tensor(n, m)
B = np.zeros(m*[n])
for i in range(n):
    B[tuple(m*[i])] = 1

n_eig = le.complex_eigen_cnt(n, m, m)
print("expecting  %d eigenvalues" % le.complex_eigen_cnt(m, m, n))
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=np.zeros((n_eig), dtype=bool),
    is_real=np.zeros((n_eig), dtype=bool))

all_eig, jcnt = le.find_all_unitary_eigenpair(
    all_eig, 0, T, B, max_itr=200, max_test=int(1e4), tol=1e-6, disc=1e-3)  

print('Total %d' % jcnt)    
check_eig(T, B, all_eig)
display(dt.display_all_real(all_eig).sort_values(by='lbd', ascending=False))



expecting  48 eigenvalues
Found 2 eigenpairs
Found 4 eigenpairs
Found 6 eigenpairs
Found 8 eigenpairs
Found 9 eigenpairs
Found 10 eigenpairs
Found 12 eigenpairs
Found 14 eigenpairs
Found 15 eigenpairs
Found 17 eigenpairs
Found 19 eigenpairs
Found 20 eigenpairs
Found 22 eigenpairs
Found 24 eigenpairs
Found 26 eigenpairs
Found 28 eigenpairs
Found 29 eigenpairs
Found 30 eigenpairs
Found 32 eigenpairs
Found 34 eigenpairs
Found 36 eigenpairs
Found 38 eigenpairs
Found 40 eigenpairs
Found 42 eigenpairs
Found 44 eigenpairs
Found 46 eigenpairs
Total 48
check lbd 45.799378
check equation 0.000000
check uniqueness
[3.33066907e-16 5.86694126e-01]


,lbd,0,1,2
19,44.311690,-0.582722,-0.555452,-0.593219
29,5.812137,0.697211,0.693637,-0.181010
9,0.406883,-0.979694,0.193923,-0.050935
8,0.386880,0.161843,-0.060296,-0.984973
28,0.330053,0.899920,0.104386,-0.423376
14,0.321326,0.951855,0.065860,-0.299391


In [13]:
n = 5
m = 3
T = ut.generate_symmetric_tensor(n, m)
B = np.zeros(m*[n])
for i in range(n):
    B[tuple(m*[i])] = 1
    
n_eig = le.complex_eigen_cnt(m, m, n)
print("expecting  %d eigenvalues" % le.complex_eigen_cnt(m, m, n))
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=np.zeros((n_eig), dtype=bool),
    is_real=np.zeros((n_eig), dtype=bool))

all_eig, jcnt = le.find_all_unitary_eigenpair(
        all_eig, 0, T, B, max_itr=200, max_test=int(1e4), tol=1e-6, disc=1e-3)
print('Total %d' % jcnt)
check_eig(T, B, all_eig)
display(dt.display_all_real(all_eig))

expecting  80 eigenvalues
Found 2 eigenpairs
Found 4 eigenpairs
Found 6 eigenpairs
Found 8 eigenpairs
Found 10 eigenpairs
Found 12 eigenpairs
Found 14 eigenpairs
Found 16 eigenpairs
Found 18 eigenpairs
Found 20 eigenpairs
Found 22 eigenpairs
Found 24 eigenpairs
Found 26 eigenpairs
Found 27 eigenpairs
Found 28 eigenpairs
Found 30 eigenpairs
Found 32 eigenpairs
Found 34 eigenpairs
Found 35 eigenpairs
Found 37 eigenpairs
Found 38 eigenpairs
Found 39 eigenpairs
Found 40 eigenpairs
Found 42 eigenpairs
Found 44 eigenpairs
Found 46 eigenpairs
Found 48 eigenpairs
Found 50 eigenpairs
Found 52 eigenpairs
Found 54 eigenpairs
Found 55 eigenpairs
Found 57 eigenpairs
Found 59 eigenpairs
Found 61 eigenpairs
Found 62 eigenpairs
Found 64 eigenpairs
Found 65 eigenpairs
Found 66 eigenpairs
Found 68 eigenpairs
Found 70 eigenpairs
Found 72 eigenpairs
Found 73 eigenpairs
Found 74 eigenpairs
Found 76 eigenpairs
Found 78 eigenpairs
Total 80
check lbd 9.187677
check equation 0.000000
check uniqueness
[2.220446

,lbd,0,1,2,3,4
26,-0.253708,0.351388,-0.249636,-0.086228,0.440109,-0.782992
27,7.621593,0.426592,0.466048,0.531334,-0.403089,0.394996
34,0.967708,0.070574,-0.234068,0.551813,0.750194,-0.270078
37,1.501340,-0.650191,0.411026,-0.534191,0.300255,-0.181100
38,-0.117940,0.341505,0.127501,-0.900308,0.230632,-0.058067
39,2.561901,0.368948,0.580031,-0.474720,0.548937,-0.027424
54,1.071516,0.714183,0.361015,0.219919,-0.299069,-0.470961
61,6.425380,-0.438754,-0.528430,-0.383941,-0.439526,0.433201
64,2.787778,0.406127,-0.192520,-0.261760,0.532605,0.667690
65,14.394390,0.468377,0.475310,0.479491,0.393675,0.412083


In [14]:
n = 5
m = 5
np.random.seed(0)

T = ut.generate_symmetric_tensor(n, m)
B = np.zeros(m*[n])
for i in range(n):
    B[tuple(m*[i])] = 1

n_eig = le.complex_eigen_cnt(n, m, m)*3//2
print("expecting  %d eigenvalues" % le.complex_eigen_cnt(m, m, n))
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=np.zeros((n_eig), dtype=bool),
    is_real=np.zeros((n_eig), dtype=bool))

all_eig, jcnt = le.find_all_unitary_eigenpair(
    all_eig, 0, T, B, max_itr=200, max_test=int(1e5), tol=1e-6, disc=1e-3)  
print('Total %d' % jcnt)
check_eig(T, B, all_eig)
display(dt.display_all_real(all_eig))

expecting  1280 eigenvalues
Found 2 eigenpairs
Found 4 eigenpairs
Found 6 eigenpairs
Found 8 eigenpairs
Found 10 eigenpairs
Found 12 eigenpairs
Found 14 eigenpairs
Found 16 eigenpairs
Found 17 eigenpairs
Found 18 eigenpairs
Found 20 eigenpairs
Found 21 eigenpairs
Found 23 eigenpairs
Found 24 eigenpairs
Found 26 eigenpairs
Found 28 eigenpairs
Found 30 eigenpairs
Found 32 eigenpairs
Found 34 eigenpairs
Found 36 eigenpairs
Found 38 eigenpairs
Found 40 eigenpairs
Found 42 eigenpairs
Found 44 eigenpairs
Found 45 eigenpairs
Found 47 eigenpairs
Found 48 eigenpairs
Found 50 eigenpairs
Found 52 eigenpairs
Found 54 eigenpairs
Found 56 eigenpairs
Found 58 eigenpairs
Found 60 eigenpairs
Found 62 eigenpairs
Found 63 eigenpairs
Found 65 eigenpairs
Found 67 eigenpairs
Found 68 eigenpairs
Found 70 eigenpairs
Found 72 eigenpairs
Found 74 eigenpairs
Found 75 eigenpairs
Found 77 eigenpairs
Found 78 eigenpairs
Found 80 eigenpairs
Found 82 eigenpairs
Found 84 eigenpairs
Found 86 eigenpairs
Found 88 eigenpa

,lbd,0,1,2,3,4
16,0.215293,0.056948,-0.098527,-0.966373,0.116879,0.198775
17,-0.852272,-0.329018,0.835079,-0.007995,-0.327769,-0.294777
20,0.120922,-0.150883,0.034777,0.268123,-0.010030,-0.950807
23,0.173836,-0.265002,-0.065729,0.398712,-0.052721,-0.873901
44,1.151459,0.012567,0.362606,0.407645,0.184317,-0.817442
47,0.134389,-0.082345,-0.307218,-0.054201,0.946349,-0.017946
62,1.003674,0.691117,0.115635,-0.501594,0.057691,0.504045
67,4.266072,-0.563156,0.425636,-0.200554,-0.678220,-0.038538
74,-0.291329,-0.931653,0.153172,0.282501,0.059973,-0.158611
77,4.608871,-0.175401,0.458502,-0.428805,0.635150,-0.414393
